# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

In [1]:
model_name = 'bert-base-uncased'#'gpt2'#
dataset = "financial_phrasebank"
subset = 'sentences_allagree'

padding_token = '[PAD]'

epochs = 3

In [2]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

per_device_train_batch_size = torch.cuda.device_count()
if per_device_train_batch_size == 0:
    per_device_train_batch_size = 1

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [3]:
## Load Dataset

In [4]:
from datasets import load_dataset

train = load_dataset(path = dataset, name = subset, split='train[:15%]')
train = train.with_format("torch")

val = load_dataset(dataset, subset, split='train[15%:20%]')
val = val.with_format("torch")

test = load_dataset(dataset, subset, split='train[20%:25%]')
test = test.with_format("torch")

print(test)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['sentence', 'label'],
    num_rows: 113
})


In [5]:
## Define Model and Tokenizer

In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer


names = train.features['label'].names
id2label = {index: item for index, item in enumerate(names)}
label2id = {item: index for index, item in enumerate(names)}

# load model
foundation_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    id2label=id2label,
    label2id=label2id,
    num_labels=len(names),
).to(device)


for param in foundation_model.parameters():
    param.requires_grad = True

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

max_length = tokenizer.model_max_length

if model_name == 'gpt2':
    tokenizer.add_special_tokens({'pad_token': padding_token})
    foundation_model.config.pad_token_id = padding_token
    foundation_model.resize_token_embeddings(len(tokenizer))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
## Tokenize Data

In [8]:
def tokenize_function(examples):
    
    outputs = tokenizer(examples["sentence"], padding="max_length",\
                                          truncation=True, return_tensors='pt',\
                                        max_length = max_length)
    return outputs

train_tokenized_datasets = train.map(
    tokenize_function,
    batched=True,
    remove_columns=['sentence']# ,train_sample.column_names
)

val_tokenized_datasets = val.map(
    tokenize_function,
    batched=True,
    remove_columns=['sentence']# ,train_sample.column_names
)

test_tokenized_datasets = test.map(
    tokenize_function,
    batched=True,
    remove_columns=['sentence']# ,train_sample.column_names
)

# train_tokenized_datasets = train_tokenized_datasets.with_format("torch")
# val_tokenized_datasets = val_tokenized_datasets.with_format("torch")
# test_tokenized_datasets = test_tokenized_datasets.with_format("torch")


Map: 100%|██████████| 113/113 [00:00<00:00, 1168.18 examples/s]


In [9]:
## Evalute Model

In [10]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [11]:
print(foundation_model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [12]:
from peft import LoraConfig

if model_name == 'gpt2':
    target_modules=["c_proj"]
elif model_name == 'bert-base-uncased':
    target_modules=["query","key","value"]



lora_config = LoraConfig(
    r=5,
    lora_alpha=16, 
    target_modules=target_modules,
    lora_dropout=0.1, 
    bias="none",
    task_type="SEQ_CLS"
)

from peft import get_peft_model
peft_model = get_peft_model(foundation_model, lora_config)
print(peft_model.print_trainable_parameters())

trainable params: 281,094 || all params: 109,763,334 || trainable%: 0.2560909820760364
None


In [13]:
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

training_args = TrainingArguments(output_dir="./peft_project", 
                                  logging_steps = 10,
                                  num_train_epochs=epochs, 
                                  learning_rate= 2e-3,
                                  per_device_train_batch_size = per_device_train_batch_size, 
                                  per_device_eval_batch_size = per_device_train_batch_size, 
                                  evaluation_strategy="epoch",
                                  save_strategy='epoch',
                                  weight_decay=0.01,
                                  load_best_model_at_end=True)


trainer = Trainer(model=peft_model, 
                  args=training_args, 
                  train_dataset=train_tokenized_datasets, 
                  eval_dataset=val_tokenized_datasets,
                  tokenizer=tokenizer,
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=compute_metrics
)

In [14]:
print("Evaluating before training...")
print("Pre-training evaluation results:", trainer.evaluate())

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Evaluating before training...


Pre-training evaluation results: {'eval_loss': 1.16550612449646, 'eval_accuracy': 0.12389380530973451, 'eval_runtime': 4.6912, 'eval_samples_per_second': 24.088, 'eval_steps_per_second': 24.088}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.077800,1.306077,0.787611
2,1.909800,1.611958,0.787611
3,0.008400,1.619907,0.787611


TrainOutput(global_step=1020, training_loss=0.7559752945428972, metrics={'train_runtime': 102.1113, 'train_samples_per_second': 9.989, 'train_steps_per_second': 9.989, 'total_flos': 269249248419840.0, 'train_loss': 0.7559752945428972, 'epoch': 3.0})

In [16]:
print("Evaluating before training...")
print("Post-training evaluation results:", trainer.evaluate())

Evaluating before training...


Post-training evaluation results: {'eval_loss': 1.306077480316162, 'eval_accuracy': 0.7876106194690266, 'eval_runtime': 4.0044, 'eval_samples_per_second': 28.219, 'eval_steps_per_second': 28.219, 'epoch': 3.0}


In [20]:
from datetime import date
filename = f"{model_name}--LORA--{date.today()}"
# trainer.save_model(filename)

peft_model.save_pretrained(filename)

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [21]:
from peft import AutoPeftModelForSequenceClassification

lora_model = AutoPeftModelForSequenceClassification.from_pretrained(filename)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: Error(s) in loading state_dict for PeftModelForSequenceClassification:
	size mismatch for base_model.model.classifier.modules_to_save.default.weight: copying a param with shape torch.Size([3, 768]) from checkpoint, the shape in current model is torch.Size([2, 768]).
	size mismatch for base_model.model.classifier.modules_to_save.default.bias: copying a param with shape torch.Size([3]) from checkpoint, the shape in current model is torch.Size([2]).

In [ ]:
test[0]

In [ ]:
inputs = tokenizer(test[0]['sentence'], return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"])
print(tokenizer.batch_decode(outputs))